In [ ]:
#@title Run this to prevent getting randomly disconnected/timed out

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

In [ ]:
#@title Install DFDNet

from pathlib import Path
!git clone https://github.com/csxmli2016/DFDNet.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
data = fin.read()
data = data.replace('if cache:', 'if False:')
fin.close()

fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
fin.write(data)
fin.close()

!pip install dlib
!pip install opencv-python
!pip install tqdm
!pip install scikit-image
!pip install dominate
!apt-get install nvidia-cuda-toolkit

print("Downloading pretrained model ...")
!wget -q --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=10poHg204q_ojLrzGShf-0XXP5yfmTW93' -O /content/DFD-complete.zip
!unzip -q -o /content/DFD-complete.zip -d /content/DFDNet
!rm /content/DFD-complete.zip

print("Done!")

In [ ]:
#@title Import files from Google Drive

Mode = "Alignments" #@param ["Alignments", "Frames"]
Archive_name = "dfdnet_test_alignments.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name
  
if Mode == "Alignments":
  zip_and_copy("/content/DFDNet/Images/alignments/to_be_enhanced/", "")
elif Mode == "Frames":
  zip_and_copy("/content/DFDNet/Images/frames/to_be_enhanced/", "")

print("Done!")

In [ ]:
#@title Get face landmarks for alignments - this must be run before enhancing alignments

!python /content/DFDNet/FaceLandmarkDetection/get_face_landmark.py

In [ ]:
#@title Run face enhancement (alignments)

!python /content/DFDNet/test_FaceDict.py

In [ ]:
#@title Run face enhancement (frames) - DOES NOT CURRENTLY WORK

!python /content/DFDNet/test_FaceDict_whole.py

In [ ]:
#@title Export to Drive { form-width: "30%" }
Mode = "alignments" #@param ["alignments", "frames"]
Archive_name = "enhanced_alignments.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "alignments":
  zip_and_copy("/content/DFDNet/Images/alignments/enhanced/facefh_dictionary/images", "fake_A")
elif Mode == "frames":
  zip_and_copy("/content/DFDNet/Images/frames", "enhanced")

In [ ]:
#@title Delete and recreate
Mode = "Delete alignment landmarks" #@param ["Delete imported alignments", "Delete enhanced alignments", "Delete alignment landmarks"]

if Mode == "Delete imported alignments":
  cmd = "rm -r /content/DFDNet/Images/alignments/to_be_enhanced ; mkdir /content/DFDNet/Images/alignments/to_be_enhanced"  
elif Mode == "Delete enhanced alignments":
  cmd = "rm -r /content/DFDNet/Images/alignments/enhanced/facefh_dictionary ; mkdir /content/DFDNet/Images/alignments/enhanced/facefh_dictionary"
elif Mode == "Delete alignment landmarks" :
  cmd = "rm -r /content/DFDNet/Images/alignments/landmarks ; mkdir /content/DFDNet/Images/alignments/landmarks"
  
!$cmd
print("Done!")